# Домашнюю работу выполнили: Фесенко Георгий, Шеин Кирилл

## Часть 1
Сперва потребовалось достать данные с сайта погоды, для чего были использованы библиотеки beautifulsoup и requests.  
Полный код приведен ниже.  
Вдохновление черпалось из следующего источника: https://habrahabr.ru/post/280238/  
Было решено взять погоду на 10 дней в городе Санкт-Петербурге.

In [181]:
import requests
from bs4 import BeautifulSoup
import pymorphy2
import pandas as pd


url = "https://www.gismeteo.ru/weather-sankt-peterburg-4079/10-days/"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}
data = requests.get(url, headers=headers).text

soup = BeautifulSoup(data, 'lxml')
weather_max_div = soup.find('div', {'class':'chart chart__temperature'}).find('div',{'class':'values'}).find_all('div',{'class':'maxt'})
weather_min_div = soup.find('div', {'class':'chart chart__temperature'}).find('div',{'class':'values'}).find_all('div',{'class':'mint'})
wind_speed_div = soup.find_all('div',{'class':'w_wind__warning w_wind__warning_'})[:10]
precipation_div = soup.find_all('div',{'class':'w_precipitation__value'})
weekdays_div = soup.find_all('div', {'class': 'w_date__day'})[:10]
wind_speed = []
max_weather = []
min_weather = []
precipation = []
weekdays = []
for el in weather_max_div:
    max_weather.append(el.text.replace('−', '-'))
for el in weather_min_div:
    min_weather.append(el.text.replace('−', '-'))
for item in wind_speed_div:
    wind_speed.append(item.text.strip())
for item in precipation_div:
    precipation.append(item.text.strip().replace(',','.'))
for item in weekdays_div:
    weekdays.append(item.text.strip().lower())

#print(max_weather)
#print(min_weather)
#print(wind_speed)
#print(precipation)
#print(weekdays)
dataP = {}
dt = '.02'
dttr = list(map(str, range(15,25)))

for (mi,ma,wi,pre,dtt,wd) in zip(min_weather,max_weather,wind_speed,precipation,dttr, weekdays):
    dataP[dtt + dt + ' (' + wd + ')'] = [mi,ma,wi,pre]

df = pd.DataFrame(dataP,index=['минимальная температура','максимальная температура','скорость ветра','уровень осадков'])
#df.to_csv('weatherForecast')

В итоге получилась такая таблица

In [231]:
df

,15.02 (чт),16.02 (пт),17.02 (сб),18.02 (вс),19.02 (пн),20.02 (вт),21.02 (ср),22.02 (чт),23.02 (пт),24.02 (сб)
минимальная температура,-9,-8,-8,-4,-6,-11,-14,-14,-7,-6
максимальная температура,-6,-6,-4,-2,-4,-7,-10,-6,-3,-1
скорость ветра,2,6,7,6,7,6,6,7,12,16
уровень осадков,0,2.6,1.4,2.2,0.3,0.1,0,0.6,0.2,1.4


## Часть 2

Мы сделали 4 вида прогнозов:

1)Средняя *температура / скорость ветра / уровень осадков* в определенный день

2)Сравнения двух дней по этим параметрам

3)*Средняя температура* за период

In [517]:
morph = pymorphy2.MorphAnalyzer()

In [518]:
#Я решил написать функции для вывода среднего отедельно для каждой позиции.
def average_temperature(n):
    min_temp = int(df[n][0])
    max_temp = int(df[n][1])
    aver_temp = (min_temp + max_temp)/2
    degrees = morph.parse('градус')[0]
    #Тут идет согласование существительного с числительным
    forecast = ' '.join([str(aver_temp),degrees.make_agree_with_number(aver_temp).word])
    return forecast


In [519]:
def average_speed(n):
    speed = int(df[n][2])
    meter = morph.parse('метр')[0]
    time = morph.parse('секунда')[0]
    in_ = 'в'
    forecast= ' '.join([str(speed),meter.make_agree_with_number(speed).word,in_,time.inflect({'accs'}).word])
    return forecast

In [520]:
def average_level(n):
    level = float(df[n][3])
    mmetr = morph.parse('миллиметр')[0]
    forecast = ' '.join([str(level),mmetr.make_agree_with_number(level).word])
    return forecast

Главная функция работает таким образом. Вводится день, когда мы хотим узнать определенный показатель и этот показатель.

Здесь и далее({*1*:'средняя температура',2:'средняя скорость ветра',3:'средний уровень осадков'})

**Все даты задаются в формате: '15.02 (чт)'**

In [521]:
def forecast_day(n,chs):
    search = {1:'температура',2:'скорость',3:'уровень'}
    city = morph.parse('Санкт-Петербург')[0]
    #Объявляем дни недели
    day_w = {'(пн)': "понедельник", '(вт)': "вторник", '(ср)': "среда",'(чт)':'четверг','(пт)':'пятница','(сб)':'суббота','(вс)':'воскресенье'}
    #Выделяем отдельно дату как число и день недели
    day_now_w = morph.parse(day_w[n.split(' ')[1]])[0]
    day_now_n = n.split(' ')[0]
    #Тут мы проводим морфологический анализ слов,которые будем использовать, чтобы мы могли из правильно склонять в дальнейшем
    average = morph.parse('cредний')[0]#0 обозначает, что мы выбираем самый вероятный разбор
    equal = morph.parse('равен')[0]
    wind = morph.parse('ветер')[0]
    level = morph.parse('осадков')[0]
    #Согласование предлогов "В" и "С"
    if   n.split(' ')[1] == '(вт)':
        start = 'Во'
    else:
        start = 'В'
    #Начальная строка и согласование названия города и дня недели    
    begin = ' '.join([start ,day_now_w.inflect({'accs'}).word,day_now_n,'в городе',(city.inflect({'loct'}).word).title()])
    search_tag = morph.parse(search[chs])[0].tag
    #Узнаем кого рода слова "характеристики"
    if 'masc' in search_tag:
        gender = 'masc'
    if 'femn' in search_tag:
        gender = 'femn'
    if search[chs] == 'температура':
        forecast = ' '.join([begin,average.inflect({gender}).word,search[chs],equal.inflect({gender}).word,average_temperature(n)])
    if search[chs] == 'скорость':
        forecast = ' '.join([begin,average.inflect({gender}).word,search[chs],wind.inflect({'gent'})[0],equal.inflect({gender}).word,average_speed(n)])
    if search[chs] == 'уровень':
        forecast = ' '.join([begin,average.inflect({gender}).word,search[chs],level.inflect({'gent'})[0],equal.inflect({gender}).word,average_level(n)])
    return forecast
    

### Показательный пример!

In [527]:
print(forecast_day('20.02 (вт)',1))

Во вторник 20.02 в городе Санкт-Петербурге cредняя температура равна -9.0 градус


# Показания за период

In [279]:
'''Здесь все тоже самое , только мы теперь сравнивае по показателям два дня. Даты вводятся в порядке возрастания'''
    aver_temp1 = (int(df[d1][0]) + int(df[d1][1]))/2
    aver_temp2 = (int(df[d2][0]) + int(df[d2][1]))/2
    deg = morph.parse('градус')[0]
    dif = aver_temp1 - aver_temp2
    if aver_temp1 > aver_temp2:
        temp = 'увеличилась на'
        forecast = ' '.join([temp,str(dif),deg.make_agree_with_number(dif).word])
        return forecast
    
    elif aver_temp1 < aver_temp2:
        temp = 'уменьшилась на'
        forecast = ' '.join([temp,str(abs(dif)),deg.make_agree_with_number(dif).word])
        return forecast
        
    else:
        temp = 'не изменилась'
        return temp         

In [300]:
print(period_temperature('24.02 (сб)','23.02 (пт)'))


увеличилась на 1.5 градусов


In [246]:
def period_speed(d1,d2):
    spd1 = int(df[d1][2])
    spd2 = int(df[d2][2])
    dif = spd1 - spd2
    meter = morph.parse('метр')[0]
    time = morph.parse('секунда')[0]
    in_ = 'в'
    if spd1 > spd2:
        spd = 'увеличилась на'
        forecast = ' '.join([spd,str(abs(dif)),meter.make_agree_with_number(dif).word,in_,time.inflect({'accs'}).word])
        return forecast
    
    elif spd1 > spd2:
        spd = 'уменьшилась на'
        forecast = ' '.join([spd,str(abs(dif)),meter.make_agree_with_number(dif).word,in_,time.inflect({'accs'}).word])
        return forecast
        
    else:
        spd = 'не изменилась'
        return spd   
        

In [301]:
print(period_speed('24.02 (сб)','23.02 (пт)'))

увеличилась на 4 метра в секунду


In [254]:
def period_level(d1,d2):
    lvl1 = float(df[d1][3])
    lvl2 = float(df[d2][3])
    mmetr = morph.parse('миллиметр')[0]
    dif = lvl1 - lvl2
    if lvl1 > lvl2:
        lvl = 'увеличилась на'
        mmetr = morph.parse('миллиметр')[0]
        forecast = ' '.join([lvl,str(dif),mmetr.make_agree_with_number(dif).word])
        return forecast
    
    elif lvl1 > lvl2:
        lvl = 'уменьшилась на'
        forecast = ' '.join([lvl,str(dif),mmetr.make_agree_with_number(dif).word])
        return forecast
        
    else:
        lvl = 'не изменилась'
        return lvl  
        

In [302]:
print(period_level('24.02 (сб)','23.02 (пт)'))

увеличилась на 1.2 миллиметров


In [324]:
def forecast_period_1(d1,d2,chs):
    search = {1:'температура',2:'скорость',3:'уровень'}
    city = morph.parse('Санкт-Петербург')[0]
    day_w = {'(пн)': "понедельник", '(вт)': "вторник", '(ср)': "среда",'(чт)':'четверг','(пт)':'пятница','(сб)':'суббота','(вс)':'воскресенье'}
    day1 = morph.parse(day_w[d1.split(' ')[1]])[0]
    day_d1_n = d1.split(' ')[0]
    day2 = morph.parse(day_w[d2.split(' ')[1]])[0]
    day_2 = day2.inflect({'ablt'}).word
    day_d2_n = d2.split(' ')[0]
    average = morph.parse('cредний')[0]
    equal = morph.parse('равен')[0]
    wind = morph.parse('ветер')[0]
    level = morph.parse('осадков')[0]
    compare = 'по сравнению'
    #Согласование предлогов "В" и "С"
    if   n.split(' ')[1] == '(вт)':
        start = 'Во'
    else:
        start = 'В'
    if d1.split(' ')[1] == '(ср)':
        wit_h = 'со'
    elif d2.split(' ')[1] == '(ср)':
        wit_h = 'со'
    else:
        wit_h = 'c'
        
    begin = ' '.join([start ,day1.inflect({'accs'}).word,day_d1_n,'в городе',(city.inflect({'loct'}).word).title()])
    search_tag = morph.parse(search[chs])[0].tag
    if 'masc' in search_tag:
        gender = 'masc'
    if 'femn' in search_tag:
        gender = 'femn'
    if search[chs] == 'температура':
        forecast_1 = ' '.join([begin,average.inflect({gender}).word,search[chs],period_temperature(d1,d2),compare,wit_h,day_2,day_d2_n])
    if search[chs] == 'скорость':
        forecast_1 = ' '.join([begin,average.inflect({gender}).word,search[chs],wind.inflect({'gent'})[0],period_temperature(d1,d2),compare,wit_h,day_2,day_d2_n])
    if search[chs] == 'уровень':
        forecast_1 = ' '.join([begin,average.inflect({gender}).word,search[chs],level.inflect({'gent'})[0],period_temperature(d1,d2),compare,wit_h,day_2,day_d2_n])
    return forecast_1
    
        

### Показательный пример!

In [325]:
'''Так как у нас нет привязки к конкретной начальной дате, то все прогнозы делаются в настоящем времене'''
'''Если бы такая была, то изменить время несложно, надо просто задать параметр TEns = "past"'''
forecast_period_1('24.02 (сб)','23.02 (пт)',2)

'В субботу 24.02 в городе Санкт-Петербурге cредняя скорость ветра увеличилась на 1.5 градусов по сравнению c пятницей 23.02'

## Средняя температура за период

Данная функция рассчитывает средние параметры за период.

Изначально мы берем значения дня каждого числа и разностью находим сколько длиться период.

Вычитаем 15, тк отчет идет не с нуля а с 15 числа.

In [531]:
def temp_average_period(d1,d2):
    n1 = int(d1.split('.')[0])
    n2 = int(d2.split('.')[0])
    deg = morph.parse('градус')[0]
    n1 = int(d1.split('.')[0])
    n2 = int(d2.split('.')[0])
    dif = (n1 - n2)+1
    ave_min = []
    ave_max = []
    for i in range(dif):
        ave_min.append(int(df.iloc[0,n2-15+i]))
        ave_max.append(int(df.iloc[1,n2-15+i]))
    temp = round(((sum(ave_min)+sum(ave_max))/dif),2)
    return ' '.join([str(temp),deg.make_agree_with_number(temp).word])

In [499]:
def spd_average_period(d1,d2):
    meter = morph.parse('метр')[0]
    time = morph.parse('секунда')[0]
    in_ = 'в'
    n1 = int(d1.split('.')[0])
    n2 = int(d2.split('.')[0])
    dif = (n1 - n2)+1
    ave = []
    for i in range(dif):
        ave.append(int(df.iloc[2,n2-15+i]))
    spd = round(((sum(ave))/dif),2)
    return ' '.join([str(spd),meter.make_agree_with_number(spd).word,in_,time.inflect({'accs'}).word])


In [503]:
def lvl_average_period(d1,d2):
    mmetr = morph.parse('миллиметр')[0]
    n1 = int(d1.split('.')[0])
    n2 = int(d2.split('.')[0])
    dif = (n1 - n2)+1
    ave = []
    for i in range(dif):
        ave.append(float(df.iloc[3,n2-15+i]))
    lvl = round(((sum(ave))/dif),2)
    return ' '.join([str(lvl),mmetr.make_agree_with_number(dif).word])

In [456]:
temp_average_period('24.02 (сб)','20.02 (пт)')

'-15.8 градусов'

In [508]:
def forecast_period_days(d1,d2,chs):
    search = {1:'температура',2:'скорость',3:'уровень'}
    city = morph.parse('Санкт-Петербург')[0]
    day_w = {'(пн)': "понедельник", '(вт)': "вторник", '(ср)': "среда",'(чт)':'четверг','(пт)':'пятница','(сб)':'суббота','(вс)':'воскресенье'}
    day1 = morph.parse(day_w[d1.split(' ')[1]])[0]
    day_d1_n = d1.split(' ')[0]
    day2 = morph.parse(day_w[d2.split(' ')[1]])[0]
    day_2 = day2.inflect({'gent'}).word
    day_1 = day1.inflect({'accs'}).word
    day_d2_n = d2.split(' ')[0]
    average = morph.parse('cредний')[0]
    equal = morph.parse('равен')[0]
    wind = morph.parse('ветер')[0]
    level = morph.parse('осадков')[0]
    period = 'за период'
    to = 'по'
    
    
    if d1.split(' ')[1] == '(ср)':
        wit_h = 'со'
    elif d2.split(' ')[1] == '(ср)':
        wit_h = 'со'
    else:
        wit_h = 'c'
        
   
    search_tag = morph.parse(search[chs])[0].tag
    if 'masc' in search_tag:
        gender = 'masc'
    if 'femn' in search_tag:
        gender = 'femn'
    if search[chs] == 'температура':
        forecast_2 = ' '.join([average.inflect({gender}).word.title(),search[chs],period,wit_h,day_2,day_d2_n,to,day_1,day_d1_n,equal.inflect({gender}).word,temp_average_period(d1,d2)])
    if search[chs] == 'скорость':
        forecast_2 = ' '.join([average.inflect({gender}).word.title(),search[chs],wind.inflect({'gent'})[0],period,wit_h,day_2,day_d2_n,to,day_1,day_d1_n,equal.inflect({gender}).word,spd_average_period(d1,d2)])
    if search[chs] == 'уровень':
        forecast_2 = ' '.join([average.inflect({gender}).word.title(),search[chs],level.inflect({'gent'})[0],period,wit_h,day_2,day_d2_n,to,day_1,day_d1_n,equal.inflect({gender}).word,lvl_average_period(d1,d2)])
    return forecast_2
    

### Показательный пример!

In [532]:
forecast_period_days('24.02 (сб)','20.02 (пт)',1)

'Cредняя температура за период c пятницы 20.02 по субботу 24.02 равна -15.8 градусов'

## Часть 3

***В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам?***  
Например в навигаторе, где голосовой помощник воспроизводит определенный текст, сгенерированный, я уверен, по шаблонам. 

Сейчас очень популярны различные чат боты, которые выдают какую-то информацию строго по определенному шаблону

***В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?***  
Любое ПО, отслеживающее определенные данные, поступающие с датчика. Пользователю будет удобнее получать данные, сгенерированные на русском, или еще каком-либо языке, чем просто видеть постоянно меняющиеся цифры. Так же эта функция будет полезна для плохо видящих людей, ведь перед тем как машина произнесет текст, этот текст надо как-то сгенерировать.

Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом.  
* Как можно разнообразить эти шаблоны?  

* Как знание синтаксической структуры предложения может помочь в этой задаче?

* Как можно разнообразить эти шаблоны?  

В данном примере мы не были привязаны к конкретной дате, если же делать реальное приложения, то в шаблон можно добавить склонение по времени. 

Также можно добавить определения рода для обращения к пользователю. Или если вы делаете рассылки на почту. 

* Как знание синтаксической структуры предложения может помочь в этой задаче?

Знания правильного порядка слов в предложениях.

Умения согласовывать словосочетания 
